# Quickstart: Semantic ranking in Azure AI Search

Azure AI Search provides secondary L2 ranking that rescores initial results using machine reading comprehension models. The L2 ranker promotes more semantically relevant matches to the top.

This is the source code for the article: [Quickstart: Semantic ranking (Python)](https://learn.microsoft.com/azure/search/search-get-started-semantic).

## Install packages and set variables

In [ ]:
! pip install -r requirements.txt --quiet

In [12]:
# Provide variables
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # Take environment variables from .env.

# The following variables from your .env file are used in this notebook
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "hotels-sample-index")

## Update an index

This code adds a semantic configuration to an existing hotels-sample-index on your search service. No search documents are deleted by this operation and the JSON definition of the index is unchanged.

In [13]:

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

# Update search schema
index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)
fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True, facetable=True, filterable=True, sortable=False),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, facetable=False, filterable=False, sortable=False, retrievable=True, analyzer_name="en.microsoft"),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.microsoft"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.microsoft"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),

        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),

        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=False),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=False, filterable=False, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String, facetable=False, filterable=False, sortable=False, analyzer_name="en.microsoft"),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False, analyzer_name="en.microsoft"),
        ])        ,
        SimpleField(name="Location", type=SearchFieldDataType.GeographyPoint, facetable=False, filterable=True, sortable=True),
        ComplexField(name="Rooms",collection=True,fields=[
                SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.microsoft"),
                SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.microsoft"),
                SearchableField(name="Type", type=SearchFieldDataType.String, analyzer_name="en.microsoft", facetable=True, filterable=True),
                SimpleField(name="BaseRate", type=SearchFieldDataType.Double, facetable=True, filterable=True),
                SearchableField(name="BedOptions", type=SearchFieldDataType.String, analyzer_name="en.microsoft", facetable=True, filterable=True),
                SimpleField(name="SleepsCount", type=SearchFieldDataType.Int64, facetable=True, filterable=True),
                SimpleField(name="SmokingAllowed", type=SearchFieldDataType.Boolean, facetable=True, filterable=True),
                SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, analyzer_name="en.microsoft", facetable=True, filterable=True)
            ]
        ),
        SimpleField(name="id", type=SearchFieldDataType.String, searchable=False, retrievable=False, facetable=False, filterable=False, sortable=False), 
        SimpleField(name="rid", type=SearchFieldDataType.String, searchable=False, retrievable=False, facetable=False, filterable=False, sortable=False)
        ]

semantic_config = SemanticConfiguration(
    name="semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)

# Specify the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

semantic_settings = SemanticSearch(configurations=[semantic_config])
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Rooms/Tags', 'Rooms/Type', 'Address/City', 'Address/Country']}]

# Update the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} updated')

 hotels-sample-index updated


## Run your first query

In [14]:
# Run an empty query (returns selected fields, all documents, no ranking, search score is uniform 1.0)
search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=credential)

results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='HotelName,Description',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 50
1.0
Grand Gaming Resort
Description: The Best Gaming Resort in the area. With elegant rooms & suites, pool, cabanas, spa, brewery & world-class gaming. This is the best place to play, stay & dine.
1.0
Country Residence Hotel
Description: All of the suites feature full-sized kitchens stocked with cookware, separate living and sleeping areas and sofa beds. Some of the larger rooms have fireplaces and patios or balconies. Experience real country hospitality in the heart of bustling Nashville. The most vibrant music scene in the world is just outside your front door.
1.0
Happy Lake Resort & Restaurant
Description: The largest year-round resort in the area offering more of everything for your vacation – at the best value! What can you enjoy while at the resort, aside from the mile-long sandy beaches of the lake? Check out our activities sure to excite both young and young-at-heart guests. We have it all, including being named “Property of the Year” and a “

## Run a term query

Other hotels in this small data set have a stronger signal for restaurant, but because "site" is a less frequent term, BM25 scores it higher.

In [16]:
# Run a text query (returns a BM25-scored result set)
results =  search_client.search(query_type='simple',
    search_text="walk to restaurants and shopping" ,
    select='HotelName,HotelId,Description',
    include_total_count=True)
    
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

7.9934134
Foot Happy Suites
Description: Downtown in the heart of the business district. Close to everything. Leave your car behind and walk to the park, shopping, and restaurants. Or grab one of our bikes and take your explorations a little further.
6.563842
Winter Panorama Resort
Description: Plenty of great skiing, outdoor ice skating, sleigh rides, tubing and snow biking. Yoga, group exercise classes and outdoor hockey are available year-round, plus numerous options for shopping as well as great spa services. Newly-renovated with large rooms, free 24-hr airport shuttle & a new restaurant. Rooms/suites offer mini-fridges & 49-inch HDTVs.
5.8222375
Uptown Chic Hotel
Description: Chic hotel near the city. High-rise hotel in downtown, within walking distance to theaters, art galleries, restaurants and shops. Visit Seattle Art Museum by day, and then head over to Benaroya Hall to catch the evening's concert performance.
4.870869
Roach Motel
Description: Perfect Location on Main Street. 

## Run a semantic query

In contrast, the L2 ranker revisits the query and uses machine reading comprehension to promote results that are more meaningful to the query.

In [17]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
    search_text="walk to restaurants and shopping", 
    select='HotelName,Description,Category', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

2.9116947650909424
Swirling Currents Hotel
Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs. 
Caption: Spacious rooms, glamorous suites and residences, rooftop pool,<em> walking access to shopping, dining, entertainment and the city center.</em> Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.

2.8783745765686035
Foot Happy Suites
Description: Downtown in the heart of the business district. Close to everything. Leave your car behind and walk to the park, shopping, and restaurants. Or grab one of our bikes and take your explorations a little further.
Caption: <em>Downtown in the heart of the business district.</em> Close to everything. <em>Leave your 

## Return semantic answers

In [18]:
# Run a semantic query that returns semantic answers  
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
 search_text="what's a good hotel for people who like to read",
 select='HotelName,Description,Category', query_caption='extractive', query_answer="extractive",)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: Nature is Home on the beach. Explore the shore by day, and then come home to our shared living space to relax around a stone fireplace, sip something warm, and explore<em> the library </em>by night. Save up to 30 percent. Valid Now through the end of the year. Restrictions and blackouts may apply.
Semantic Answer Score: 0.9890000224113464

2.192565441131592
Stay-Kay City Hotel
Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Caption: This<em> classic </em>hotel is<em> fully-refurbished </em>and ideally located on the<em> main commercial artery of the </em>city in the<em> heart of New York.</em> A few minutes away is Times Square and the historic centre of the city, as well as other places of inter

## Clean up

If you're finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [ ]:
try:
    result = index_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Confirm the index deletion by running the following script that returns the index definition. If hotels-sample-index isn't returned, you've successfully deleted the index and have completed this quickstart.

In [ ]:
try:
    result = index_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)
